In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/salesman',True)

Out[1]: True

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/order',True)

Out[2]: True

In [0]:
%sql
CREATE TABLE salesman(
  salesman_id INT,
  name VARCHAR(20),
  city VARCHAR(20),
  commission DECIMAL(5,2)
)USING DELTA;

In [0]:
%sql
INSERT INTO salesman VALUES(5001,'James Hoog','New York',0.15);
INSERT INTO salesman VALUES(5002,'Nail Knite','Paris',0.13);
INSERT INTO salesman VALUES(5005,'Pit Alex','London',0.11);
INSERT INTO salesman VALUES(5006,'Mc Lyon','Paris',0.14);
INSERT INTO salesman VALUES(5003,'Lauson Hen','San Jose',0.12);
INSERT INTO salesman VALUES(5007,'Paul Adam','Rome',0.13);


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
CREATE TABLE order(
  ord_no INT,
  purch_amt DECIMAL(6,2),
  ord_date DATE,
  customer_id INT,
  salesman_id INT
);

In [0]:
%sql
INSERT INTO order VALUES (70001,150.5,'2012-10-05',3005,5002);
INSERT INTO order VALUES (70009,270.65,'2012-09-10',3001,5005);
INSERT INTO order VALUES (70002,65.26,'2012-10-05',3002,5001);
INSERT INTO order VALUES (70004,110.5,'2012-08-17',3009,5003);
INSERT INTO order VALUES (70007,948.5,'2012-09-10',3005,5002);
INSERT INTO order VALUES (70005,2400.6,'2012-07-27',3007,5001);
INSERT INTO order VALUES (70008,5760,'2012-09-10',3002,5001);
INSERT INTO order VALUES (70010,1983.43,'2012-10-10',3004,5006);
INSERT INTO order VALUES (70003,2480.4,'2012-10-10',3009,5003);
INSERT INTO order VALUES (70012,250.45,'2012-06-27',3008,5002);
INSERT INTO order VALUES (70011,75.29,'2012-08-17',3003,5007);
INSERT INTO order VALUES (70013,3045.6,'2012-04-25',3002,5001);

num_affected_rows,num_inserted_rows
1,1


In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/customers',True)

Out[8]: True

In [0]:
%sql
CREATE TABLE customers(
  customer_id INT,
  cust_name varchar(20),
  city VARCHAR(20),
  grade INT,
  salesman_id INT
);

In [0]:
%sql
INSERT INTO customers VALUES(3002,'Nick Rimando', 'New York'  ,'100','5001');
INSERT INTO customers VALUES(3005,'Graham Zusi' , 'California','200','5002');
INSERT INTO customers VALUES(3001,'Brad Guzan'  , 'London'   ,'100','5005');
INSERT INTO customers VALUES(3004,'Fabian Johns', 'Paris'    ,'300','5006');
INSERT INTO customers VALUES(3007,'Brad Davis' , 'New York' ,'200','5001');
INSERT INTO customers VALUES(3009,'Geoff Camero', 'Berlin'    ,'100','5003');
INSERT INTO customers VALUES(3008,'Julian Green', 'London'   ,'300','5002');
INSERT INTO customers VALUES(3003,'Jozy Altidor', 'Moncow'    ,'200','5007');

num_affected_rows,num_inserted_rows
1,1


1. From the following tables, write a SQL query to find all the orders issued by the salesman 'Paul Adam'. Return ord_no, purch_amt, ord_date, customer_id and salesman_id.



In [0]:
%sql
select * from order 
where salesman_id in(
  select salesman_id from salesman
  where name = 'Paul Adam'
);

ord_no,purch_amt,ord_date,customer_id,salesman_id
70011,75.29,2012-08-17,3003,5007


2. From the following tables write a SQL query to find all orders generated by London-based salespeople. Return ord_no, purch_amt, ord_date, customer_id, salesman_id.

In [0]:
%sql
select * from order
where salesman_id in(
  select salesman_id from salesman
  where city = 'London'
);

ord_no,purch_amt,ord_date,customer_id,salesman_id
70009,270.65,2012-09-10,3001,5005


3. From the following tables write a SQL query to find all orders generated by the salespeople who may work for customers whose id is 3007. Return ord_no, purch_amt, ord_date, customer_id, salesman_id.

In [0]:
%sql
select * from order
where salesman_id in (
  select distinct salesman_id from order
  where customer_id = 3007
);

ord_no,purch_amt,ord_date,customer_id,salesman_id
70013,3045.60,2012-04-25,3002,5001
70008,5760.00,2012-09-10,3002,5001
70005,2400.60,2012-07-27,3007,5001
70002,65.26,2012-10-05,3002,5001


4. From the following tables write a SQL query to find the order values greater than the average order value of 10th October 2012. Return ord_no, purch_amt, ord_date, customer_id, salesman_id.

In [0]:
%sql
SELECT * FROM order 
WHERE purch_amt >(
  SELECT AVG(purch_amt) FROM order
  WHERE ord_date = '2012-10-10'
);

ord_no,purch_amt,ord_date,customer_id,salesman_id
70013,3045.60,2012-04-25,3002,5001
70008,5760.00,2012-09-10,3002,5001
70003,2480.40,2012-10-10,3009,5003
70005,2400.60,2012-07-27,3007,5001


5. From the following tables, write a SQL query to find all the orders generated in New York city. Return ord_no, purch_amt, ord_date, customer_id and salesman_id.

In [0]:
%sql
SELECT * FROM order
WHERE salesman_id IN (
  SELECT salesman_id FROM salesman
  WHERE city = 'New York'
);

ord_no,purch_amt,ord_date,customer_id,salesman_id
70013,3045.60,2012-04-25,3002,5001
70008,5760.00,2012-09-10,3002,5001
70002,65.26,2012-10-05,3002,5001
70005,2400.60,2012-07-27,3007,5001


6. From the following tables write a SQL query to determine the commission of the salespeople in Paris. Return commission.



In [0]:
%sql
SELECT commission FROM salesman
WHERE salesman_id IN (
SELECT salesman_id FROM customers 
WHERE city = 'Paris' 
);


commission
0.14


7. Write a query to display all the customers whose ID is 2001 below the salesperson ID of Mc Lyon.

In [0]:
%sql
SELECT * FROM customers
WHERE customer_id + 2001 =
(
  SELECT salesman_id AS s_id FROM salesman
  WHERE name = "Mc Lyon"
)

customer_id,cust_name,city,grade,salesman_id
3005,Graham Zusi,California,200,5002


8. From the following tables write a SQL query to count the number of customers with grades above the average in New York City. Return grade and count.  



In [0]:
%sql
SELECT grade,COUNT(grade) FROM customers
WHERE grade > 
(
  SELECT AVG(grade) FROM customers 
  WHERE city = 'New York'
)
GROUP BY grade;

grade,count(grade)
200,3
300,2


9. From the following tables, write a SQL query to find those salespeople who earned the maximum commission. Return ord_no, purch_amt, ord_date, and salesman_id.

In [0]:
%sql
SELECT * FROM order
WHERE salesman_id IN (
  SELECT salesman_id FROM salesman
  WHERE commission = (
    SELECT MAX(commission) FROM salesman
  )
);

ord_no,purch_amt,ord_date,customer_id,salesman_id
70013,3045.60,2012-04-25,3002,5001
70008,5760.00,2012-09-10,3002,5001
70002,65.26,2012-10-05,3002,5001
70005,2400.60,2012-07-27,3007,5001


10. From the following tables write SQL query to find the customers who placed orders on 17th August 2012. Return ord_no, purch_amt, ord_date, customer_id, salesman_id and cust_name.



In [0]:
%sql
SELECT o.ord_no, o.purch_amt, o.ord_date, o.customer_id, o.salesman_id, c.cust_name FROM order o
INNER JOIN customers c
ON o.customer_id = c.customer_id
WHERE o.ord_date = '2012-08-17';

ord_no,purch_amt,ord_date,customer_id,salesman_id,cust_name
70011,75.29,2012-08-17,3003,5007,Jozy Altidor
70004,110.50,2012-08-17,3009,5003,Geoff Camero


11. From the following tables write a SQL query to find salespeople who had more than one customer. Return salesman_id and name.



In [0]:
%sql
SELECT Salesman_id,name FROM salesman
WHERE salesman_id IN (
  SELECT salesman_id FROM customers
  GROUP BY salesman_id
  HAVING COUNT(DISTINCT customer_id) > 1
);

Salesman_id,name
5001,James Hoog
5002,Nail Knite


12. From the following tables write a SQL query to find those orders, which are higher than the average amount of the orders. Return ord_no, purch_amt, ord_date, customer_id and salesman_id.

In [0]:
%sql
SELECT ord_no, purch_amt, ord_date, customer_id, salesman_id FROM order a
WHERE purch_amt >
(
  SELECT AVG(Purch_amt) FROM order b
  WHERE a.customer_id = b.customer_id
);


ord_no,purch_amt,ord_date,customer_id,salesman_id
70013,3045.60,2012-04-25,3002,5001
70008,5760.00,2012-09-10,3002,5001
70003,2480.40,2012-10-10,3009,5003
70007,948.50,2012-09-10,3005,5002


13. From the following tables write a SQL query to find those orders that are equal or higher than the average amount of the orders. Return ord_no, purch_amt, ord_date, customer_id and salesman_id.

In [0]:
%sql
SELECT ord_no, purch_amt, ord_date, customer_id,salesman_id FROM order a
WHERE purch_amt >= (
    SELECT AVG(purch_amt) FROM order b
    WHERE a.customer_id = b.customer_id
);

ord_no,purch_amt,ord_date,customer_id,salesman_id
70013,3045.60,2012-04-25,3002,5001
70008,5760.00,2012-09-10,3002,5001
70003,2480.40,2012-10-10,3009,5003
70011,75.29,2012-08-17,3003,5007
70010,1983.43,2012-10-10,3004,5006
70009,270.65,2012-09-10,3001,5005
70005,2400.60,2012-07-27,3007,5001
70012,250.45,2012-06-27,3008,5002
70007,948.50,2012-09-10,3005,5002


14. Write a query to find the sums of the amounts from the orders table, grouped by date, and eliminate all dates where the sum was not at least 1000.00 above the maximum order amount for that date.

In [0]:
%sql
SELECT ord_date,sum_purch_amt FROM(
  SELECT ord_date,SUM(purch_amt) sum_purch_amt,MAX(purch_amt) AS max_purch_amt FROM order
GROUP BY ord_date
HAVING SUM(purch_amt) > max_purch_amt + 1000
);

ord_date,sum_purch_amt
2012-09-10,6979.15
2012-10-10,4463.83


15. Write a query to extract all data from the customer table if and only if one or more of the customers in the customer table are located in London. 

In [0]:
%sql
SELECT * FROM customers
WHERE EXISTS (
  SELECT * FROM customers 
  WHERE city = 'London'
);

customer_id,cust_name,city,grade,salesman_id
3005,Graham Zusi,California,200,5002
3002,Nick Rimando,New York,100,5001
3007,Brad Davis,New York,200,5001
3003,Jozy Altidor,Moncow,200,5007
3009,Geoff Camero,Berlin,100,5003
3008,Julian Green,London,300,5002
3004,Fabian Johns,Paris,300,5006
3001,Brad Guzan,London,100,5005


16. From the following tables write a SQL query to find salespeople who deal with multiple customers. Return salesman_id, name, city and commission.

In [0]:
%sql
SELECT * FROM salesman
WHERE salesman_id IN (
  SELECT salesman_id FROM customers
GROUP BY salesman_id
HAVING COUNT(DISTINCT customer_id) > 1
);

salesman_id,name,city,commission
5001,James Hoog,New York,0.15
5002,Nail Knite,Paris,0.13


17. From the following tables write a SQL query to find salespeople who deal with a single customer. Return salesman_id, name, city and commission.

In [0]:
%sql
SELECT * FROM salesman
WHERE salesman_id IN (
  SELECT DISTINCT salesman_id FROM customers a
  WHERE  NOT EXISTS (
    SELECT * FROM customers b
    WHERE b.salesman_id = a.salesman_id
    AND b.customer_id <> a.customer_id
  )
);

salesman_id,name,city,commission
5003,Lauson Hen,San Jose,0.12
5005,Pit Alex,London,0.11
5007,Paul Adam,Rome,0.13
5006,Mc Lyon,Paris,0.14


18. From the following tables, write a SQL query to find the salespeople who deal the customers with more than one order. Return salesman_id, name, city and commission.

In [0]:
%sql
SELECT * FROM salesman
WHERE salesman_id IN (
  SELECT salesman_id FROM customers
  WHERE customer_id IN (
    SELECT customer_id FROM order
    GROUP BY customer_id
    HAVING COUNT(*) > 1
  )
);

salesman_id,name,city,commission
5003,Lauson Hen,San Jose,0.12
5001,James Hoog,New York,0.15
5002,Nail Knite,Paris,0.13


19. From the following tables write a SQL query to find all salespeople who are located in any city where there is at least one customer. Return salesman_id, name, city and commission.

In [0]:
%sql
SELECT * FROM salesman a 
WHERE EXISTS (
  SELECT salesman_id FROM customers b
  WHERE a.city = b.city);

salesman_id,name,city,commission
5001,James Hoog,New York,0.15
5002,Nail Knite,Paris,0.13
5005,Pit Alex,London,0.11
5006,Mc Lyon,Paris,0.14


20. From the following tables write a SQL query to find salespeople whose place of residence matches any city where customers live. Return salesman_id, name, city and commission.



In [0]:
%sql
SELECT * FROM salesman
WHERE city IN (
  SELECT city FROM customers
);

salesman_id,name,city,commission
5001,James Hoog,New York,0.15
5002,Nail Knite,Paris,0.13
5005,Pit Alex,London,0.11
5006,Mc Lyon,Paris,0.14


21. From the following tables write a SQL query to find all those salespeople whose names appear alphabetically lower than the customer’s name. Return salesman_id, name, city, commission.

In [0]:
%sql
SELECT * FROM salesman a
WHERE EXISTS(
  SELECT * FROM customers b
  WHERE a.name < b.cust_name
);

salesman_id,name,city,commission
5003,Lauson Hen,San Jose,0.12
5001,James Hoog,New York,0.15
5002,Nail Knite,Paris,0.13
5006,Mc Lyon,Paris,0.14


22. From the following table write a SQL query to find all those customers with a higher grade than all the customers alphabetically below the city of New York. Return customer_id, cust_name, city, grade, salesman_id.

In [0]:
%sql
SELECT * FROM customers
  WHERE grade > ANY(
    SELECT grade FROM customers 
    WHERE city < 'New York'
  );

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-2361588308929772>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2361588308929772>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUICogRlJPTSBjdXN0b21lcnMoCiAgV0hFUkUgZ3JhZGUgPiBBTlkoCiAgICBTRUxFQ1QgZ3JhZGUgRlJPTSBjdXN0b21lcnMgCiAgICBXSEVSRSBjaXR5IDwgJ05ldyBZb3JrJwogICkKKQ==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name,

23. From the following table write a SQL query to find all those orders whose order amount exceeds at least one of the orders placed on September 10th 2012. Return ord_no, purch_amt, ord_date, customer_id and salesman_id.

In [0]:
%sql
SELECT * FROM order
WHERE purch_amt > ANY (
  SELECT purch_amt FROM order
  WHERE ord_date = '2012-09-10');

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-2361588308929775>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2361588308929775>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUICogRlJPTSBvcmRlcgpXSEVSRSBFWElTVFMgKAogIFNFTEVDVCBvcmRfbm8gRlJPTSBvcmRlcgogIFdIRVJFIHB1cmNoX2FtdCA+IChTRUxFQ1QgcHVyY2hfYW10IEZST00gb3JkZXIgV0hFUkUgb3JkX2RhdGUgPSAnMjAxMi0wOS0xMCcpCik=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.l

24. From the following tables write a SQL query to find orders where the order amount is less than the order amount of a customer residing in London City. Return ord_no, purch_amt, ord_date, customer_id and salesman_id.

In [0]:
%sql
SELECT * FROM order 
WHERE purch_amt < ANY(SELECT purch_amt
	FROM order a, customers b
	WHERE  a.customer_id = b.customer_id
	AND b.city = 'London')
;

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-2361588308929778>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2361588308929778>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUICogRlJPTSBvcmRlcgpXSEVSRSBwdXJjaF9hbXQgPCgKICBTRUxFQ1QgcHVyY2hfYW10ICBGUk9NIG9yZGVyCiAgV0hFUkUgY3VzdG9tZXJfaWQgSU4gKAogICAgU0VMRUNUIGN1c3RvbWVyX2lkIEZST00gY3VzdG9tZXJzIFdIRVJFIGNpdHkgPSAnTG9uZG9uJwogICkKKQ==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwar

25. From the following tables write a SQL query to find those orders where every order amount is less than the maximum order amount of a customer who lives in London City. Return ord_no, purch_amt, ord_date, customer_id and salesman_id.



In [0]:
%sql
SELECT * FROM order 
WHERE purch_amt < (
  SELECT MAX(purch_amt) FROM order o, customers c
  WHERE o.customer_id = c.customer_id
  AND c.city = 'London'
);

ord_no,purch_amt,ord_date,customer_id,salesman_id
70011,75.29,2012-08-17,3003,5007
70002,65.26,2012-10-05,3002,5001
70012,250.45,2012-06-27,3008,5002
70004,110.50,2012-08-17,3009,5003
70001,150.50,2012-10-05,3005,5002


26. From the following tables write a SQL query to find those customers whose grades are higher than those living in New York City. Return customer_id, cust_name, city, grade and salesman_id.



In [0]:
%sql
SELECT * FROM customers 
WHERE grade > (SELECT grade FROM customers 
WHERE city = 'New York');


  


org.apache.spark.SparkException: [SCALAR_SUBQUERY_TOO_MANY_ROWS] More than one row returned by a subquery used as an expression.
== SQL(line 2, position 15) ==
WHERE grade > (SELECT grade FROM customers 
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
WHERE city = 'New York')
^^^^^^^^^^^^^^^^^^^^^^^^

	at org.apache.spark.sql.errors.QueryExecutionErrors$.multipleRowSubqueryError(QueryExecutionErrors.scala:2944)
	at org.apache.spark.sql.execution.ScalarSubquery.updateResult(subquery.scala:87)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$waitForSubqueries$1(SparkPlan.scala:377)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$waitForSubqueries$1$adapted(SparkPlan.scala:376)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.sql.execution.SparkPlan.waitForSubqueries(SparkPlan.scala:376)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:333)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:330)
	at org.apache.spark.sql.execution.CodegenSupport.produce(WholeStageCodegenExec.scala:93)
	at org.apache.spark.sql.execution.CodegenSupport.produce$(WholeStageCodegenExec.scala:93)
	at org.apache.spark.sql.execution.FilterExec.produce(basicPhysicalOperators.scala:222)
	at org.apache.spark.sql.execution.WholeStageCodegenExec.doCodeGen(WholeStageCodegenExec.scala:661)
	at org.apache.spark.sql.execution.WholeStageCodegenExec.doExecute(WholeStageCodegenExec.scala:724)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:277)
	at org.apache.spark.sql.execution.SparkPlan$.org$apache$spark$sql$execution$SparkPlan$$withExecuteQueryLogging(SparkPlan.scala:108)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:334)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:330)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:273)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:123)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:135)
	at org.apache.spark.sql.execution.qrc.InternalRowFormat$.collect(cachedSparkResults.scala:122)
	at org.apache.spark.sql.execution.qrc.InternalRowFormat$.collect(cachedSparkResults.scala:110)
	at org.apache.spark.sql.execution.qrc.InternalRowFormat$.collect(cachedSparkResults.scala:92)
	at org.apache.spark.sql.execution.qrc.ResultCacheManager.$anonfun$computeResult$1(ResultCacheManager.scala:537)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.execution.qrc.ResultCacheManager.collectResult$1(ResultCacheManager.scala:529)
	at org.apache.spark.sql.execution.qrc.ResultCacheManager.$anonfun$computeResult$2(ResultCacheManager.scala:547)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec.$anonfun$withFinalPlanUpdate$1(AdaptiveSparkPlanExec.scala:634)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec.withFinalPlanUpdate(AdaptiveSparkPlanExec.scala:632)
	at org.apache.spark.sql.execution.qrc.ResultCacheManager.computeResult(ResultCacheManager.scala:547)
	at org.apache.spark.sql.execution.qrc.ResultCacheManager.$anonfun$getOrComputeResultInternal$1(ResultCacheManager.scala:402)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.qrc.ResultCacheManager.getOrComputeResultInternal(ResultCacheManager.scala:395)
	at org.apache.spark.sql.execution.qrc.ResultCacheManager.getOrComputeResult(ResultCacheManager.scala:289)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect